In [621]:
import pandas as pd

## Import datasets and keep relevant columns

In [601]:
# Import datasets
df_clinic = pd.read_csv("./Data Sets/clinic.csv")
print(len(df_clinic))
df_household = pd.read_csv("./Data Sets/household.csv")
print(len(df_household))

6992
1084


In [602]:
# Keep relevant columns
df_clinic = df_clinic[['pid', 'sex', 'site', 'enrollment_date', 'enrollment_age', 'enrollment_weight',
                       'discharge_date', 'discharge_age', 'discharge_weight', 'staffmember']]
df_household = df_household[['pid', 'c_sex', 'todate', 'weight', 'site']]

## Check for missing values and duplicates

In [579]:
# Check for missing values

In [603]:
df_clinic[df_clinic.isna().any(axis=1)]

,pid,sex,site,enrollment_date,enrollment_age,enrollment_weight,discharge_date,discharge_age,discharge_weight,staffmember


In [604]:
df_household[df_household.isna().any(axis=1)]

,pid,c_sex,todate,weight,site
104,24-3861,f,11/21/2024,NaN,galdamari
650,24-2293,f,8/5/2024,NaN,galdamari
1048,24-2293,f,8/5/2024,NaN,galdamari


In [605]:
# Delete rows with missing values
df_household= df_household.dropna()
print(len(df_household))

1081


In [606]:
# Check for duplicate values and remove any duplicates.
# There are some patients with multiple records.
# Some records are exact duplicates, while others have different dates and weights for the same patient (pid).
# We should check pid 24-2374, 24-2312, 24-3829 for further analysis.
# For simplicity, I just have deleted duplicate records based on pid, keeping only the last occurrence for each patient.
df_clinic = df_clinic[df_clinic.duplicated('pid', keep='last') | ~df_clinic.duplicated('pid', keep=False)]
print(len(df_clinic))
df_household = df_household[df_household.duplicated('pid', keep='last') | ~df_household.duplicated('pid', keep=False)]
print(len(df_household))

6982
1043


## Check each variable

### 1. clinic site variable

In [607]:
# Check the clinic site variable
print(df_clinic['site'].unique())
print(df_household['site'].unique())

['galdamari' 'kwami' 'birin_fulani' 'dukku' 'bani_gaye' 'lergo_abba'
 'wuro_modibbo' 'dadin_kowa' 'akko']
['bani_gaye' 'birin_fulani' 'dukku' 'kwami' 'galdamari']


In [608]:
# As data description mentioned that it contains five clinics, I will just keep the data with clinic sites of
# ['bani_gaye' 'birin_fulani' 'dukku' 'kwami' 'galdamari']
df_clinic = df_clinic[df_clinic['site'].isin(list(df_household['site'].unique()))]

### 2. sex variable

In [609]:
# Check the sex variable
print(df_clinic['sex'].unique())
print(df_household['c_sex'].unique())

['male' 'female' 'MALE' 'FEMALE' 'm' 'M' 'f' 'F' 'Male' 'Female']
['f' 'm']


In [610]:
# Unify the terms for female and male
df_clinic = df_clinic.replace(to_replace=['male', 'MALE', 'M', 'Male'],value='m')
df_clinic = df_clinic.replace(to_replace=['female', 'FEMALE', 'F', 'Female'],value='f')

### 3. weight variables: [clinic] enrollment_weight, discharge_weight / [household] weight

In [611]:
# Check for negative values
df_clinic[(df_clinic['enrollment_weight']<=0)|(df_clinic['discharge_weight']<=0)].head(5)

,pid,sex,site,enrollment_date,enrollment_age,enrollment_weight,discharge_date,discharge_age,discharge_weight,staffmember
547,24-2680,m,kwami,7/4/2024,3,-1000000.00,7/25/2024,4,3.95,staff20
863,24-1004,m,kwami,3/11/2024,18,7.00,4/26/2024,19,0.00,staff20
871,24-1021,f,kwami,3/14/2024,17,7.25,4/25/2024,18,0.00,staff20
2240,24-2813,m,kwami,7/19/2024,48,11.60,8/23/2024,49,0.00,staff20
2350,24-4050,f,kwami,10/1/2024,22,6.45,12/24/2024,25,0.00,staff7


In [612]:
df_household[(df_household['weight']<=0)]

,pid,c_sex,todate,weight,site
30,24-4479,m,1/10/2025,-10.25,bani_gaye
493,24-2249,m,10/2/2024,-3.75,birin_fulani
935,24-2938,m,8/20/2024,-4.15,birin_fulani


In [613]:
# Delete rows with negative values in the weight variable: These values are likely errors.
# We can consider imputing them later if needed.
df_clinic = df_clinic[(df_clinic['enrollment_weight']>0)&(df_clinic['discharge_weight']>0)]
print(len(df_clinic))
df_household = df_household[(df_household['weight']>0)]
print(len(df_household))

6888
1040


In [614]:
# Check for outliers
df_clinic[(df_clinic['enrollment_weight']>50)|(df_clinic['discharge_weight']>50)]

,pid,sex,site,enrollment_date,enrollment_age,enrollment_weight,discharge_date,discharge_age,discharge_weight,staffmember
93,24-3122,m,kwami,8/5/2024,8,4.50,11/1/2024,11,51.001493,staff23
95,24-3871,m,birin_fulani,9/16/2024,21,7.45,11/1/2024,22,78.479040,staff5
244,24-3878,f,birin_fulani,9/17/2024,21,6.45,11/12/2024,23,71.504480,staff5
963,23-2952,f,galdamari,12/27/2023,11,4.55,3/28/2024,14,74.924377,staff9
977,24-0520,f,bani_gaye,2/12/2024,13,6.65,3/25/2024,14,75.413996,staff4
1453,23-2167,m,bani_gaye,10/16/2023,21,7.10,11/21/2023,22,51.001493,staff2
2142,24-2695,f,kwami,7/8/2024,8,4.00,9/24/2024,11,97.915537,staff17
2281,24-2481,f,birin_fulani,6/25/2024,12,5.60,8/13/2024,14,52.328854,staff6
2582,24-4545,f,birin_fulani,10/29/2024,12,5.80,12/31/2024,14,97.915537,staff5
2697,24-1428,m,galdamari,4/17/2024,14,6.30,6/5/2024,16,75.413996,staff2


In [615]:
df_household[(df_household['weight']>50)]

,pid,c_sex,todate,weight,site
705,24-2977,m,9/18/2024,708.7,birin_fulani


In [616]:
# If the weight is over 50, it might be due to a decimal point error. Divide the values by 10 to correct them.
df_clinic.loc[(df_clinic['enrollment_weight'] > 50) | (df_clinic['discharge_weight'] > 50), 
              ['discharge_weight']] /= 10
df_household.loc[(df_household['weight'] > 50), 
              ['weight']] /= 10

### 4. age variables: [clinic] enrollment_age, discharge_age

In [617]:
# Check for negative values
df_clinic[(df_clinic['enrollment_age']<=0)|(df_clinic['discharge_age']<=0)].head(5)

,pid,sex,site,enrollment_date,enrollment_age,enrollment_weight,discharge_date,discharge_age,discharge_weight,staffmember


In [618]:
# Check for outliers
df_clinic[(df_clinic['enrollment_age']>60)|(df_clinic['discharge_age']>60)].head(5)

,pid,sex,site,enrollment_date,enrollment_age,enrollment_weight,discharge_date,discharge_age,discharge_weight,staffmember


### 5. staff member variable: [clinic] staffmember

In [619]:
# Keep only the number
df_clinic['staffmember'] = df_clinic['staffmember'].replace(r'^staff(\d+)$', r'\1', regex=True)

In [620]:
# save the cleaned dataset
df_clinic.to_csv("./Data Sets/clinic_cleaned.csv")
df_household.to_csv("./Data Sets/household_cleaned.csv")